## PRINCE KUMAR

## Real - time - face - mask - detection

### Data Preprocessing

In [19]:
import cv2,os

data_path='dataset/train'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels))7

print(label_dict)
print(categories)
print(labels)

{'without_mask': 0, 'with_mask': 1}
['without_mask', 'with_mask']
[0, 1]


In [9]:
img_size=100
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            #Coverting the image into gray scale
            resized=cv2.resize(gray,(img_size,img_size))
            #resizing the gray scale into 100x100, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)

        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

In [10]:
import numpy as np

data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)

from keras.utils import np_utils

new_target=np_utils.to_categorical(target)

In [11]:
np.save('data',data)
np.save('target',new_target)


## Convolutional Neural Network Architecture

In [12]:
import numpy as np

data=np.load('data.npy')
target=np.load('target.npy')

In [13]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [14]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [15]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=10,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/10
5/5 [==============================] - 19s 964ms/step - loss: 0.8746 - accuracy: 0.4936 - val_loss: 0.6929 - val_accuracy: 0.4571
INFO:tensorflow:Assets written to: model-001.model\assets
Epoch 2/10
5/5 [==============================] - 4s 878ms/step - loss: 0.6921 - accuracy: 0.5199 - val_loss: 0.6915 - val_accuracy: 0.4571
INFO:tensorflow:Assets written to: model-002.model\assets
Epoch 3/10
5/5 [==============================] - 4s 868ms/step - loss: 0.6874 - accuracy: 0.5140 - val_loss: 0.6858 - val_accuracy: 0.4571
INFO:tensorflow:Assets written to: model-003.model\assets
Epoch 4/10
5/5 [==============================] - 4s 866ms/step - loss: 0.6533 - accuracy: 0.5920 - val_loss: 0.6651 - val_accuracy: 0.5714
INFO:tensorflow:Assets written to: model-004.model\assets
Epoch 5/10
5/5 [==============================] - 4s 813ms/step - loss: 0.6086 - accuracy: 0.6838 - val_loss: 0.6952 - val_accuracy: 0.4857
Epoch 6/10
5/5 [==============================] - 4s 866ms/step - l

## Detecting Faces 

In [16]:
from keras.models import load_model
import cv2
import numpy as np

In [20]:
model = load_model('model-017.model')

face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)

labels_dict={0:'WITH MASK',1:'WITHOUT MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [21]:

while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

    for x,y,w,h in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))78
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()